In [2]:
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage
from langchain_core.language_models import BaseLanguageModel
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

langchain_openai_model: BaseLanguageModel[BaseMessage] = ChatOpenAI(model="gpt-4o-mini", temperature=0, openai_api_key=os.getenv("OPENAI_API_KEY"))

evaluator_llm = LangchainLLMWrapper(langchain_llm=langchain_openai_model)

In [3]:
from langchain_openai import OpenAIEmbeddings
from ragas.embeddings import LangchainEmbeddingsWrapper

openai_embeddings = OpenAIEmbeddings(api_key=api_key)

evaluator_embeddings = LangchainEmbeddingsWrapper(openai_embeddings)

In [9]:
from ragas.metrics import Faithfulness,ResponseRelevancy,LLMContextPrecisionWithoutReference,LLMContextRecall,NoiseSensitivity
from ragas.dataset_schema import SingleTurnSample

sample = SingleTurnSample(
    user_input="Obat apa yang cocok untuk Penyakit gangguan kecemasan umum ?",
    response="Obat yang dapat direkomendasikan adalah Sertraline, dengan tingkat risiko sedang. Obat ini cocok untuk mengatasi gejala seperti merasa gelisah, mudah tersinggung, dan kesulitan berkonsentrasi. Dosis yang disarankan adalah 50 mg per hari, yang bisa dikonsumsi sekali sehari, baik pada pagi maupun malam hari. Efek samping yang mungkin timbul meliputi kesulitan tidur, kelelahan, dan gangguan konsentrasi.",
    reference="Sertraline adalah obat antidepresan golongan Selective Serotonin Reuptake Inhibitor (SSRI) yang bekerja dengan cara menyeimbangkan kadar serotonin dalam otak, sehingga membantu mengatur suasana hati. Obat ini umum digunakan untuk mengatasi depresi, gangguan obsesif kompulsif (OCD), gangguan panik, PTSD, dan gangguan kecemasan sosial. Dosis awal untuk dewasa umumnya 50 mg sekali sehari, yang bisa ditingkatkan bertahap hingga maksimal 200 mg per hari tergantung kondisi dan respon pasien. Efek samping yang mungkin timbul meliputi kesulitan tidur, kelelahan, dan gangguan konsentrasi. Penggunaan Sertraline perlu dihindari jika sedang menggunakan MAOI, memiliki riwayat gangguan bipolar, penyakit jantung, atau sedang hamil dan menyusui, kecuali atas rekomendasi dokter. Sertraline juga tidak disarankan untuk dikonsumsi bersama alkohol dan harus digunakan dengan pengawasan medis. Beberapa merek dagang dari Sertraline termasuk Zoloft, Zerlin, dan Serlof. Obat ini tersedia dalam bentuk tablet dan hanya boleh digunakan sesuai resep dokter.",
    retrieved_contexts = [
        "Sertraline adalah obat antidepresan golongan SSRI yang digunakan untuk menangani depresi, OCD, gangguan panik, PTSD, gangguan kecemasan sosial, dan premenstrual dysphoric disorder.",
        "Sertraline bekerja dengan menyeimbangkan kadar serotonin di otak yang memengaruhi suasana hati dan emosi.",
        "Merek dagang Sertraline antara lain Deptral, Fatral, Fridep 50, Iglodep, Nudep, Serlof, Sernade, Sertraline HCL, Zerlin, dan Zoloft.",
        "Obat ini dikonsumsi oleh dewasa dan anak usia ≥6 tahun (untuk OCD) dengan pengawasan dokter.",
        "Sertraline masuk kategori C untuk ibu hamil, artinya risiko terhadap janin ada, namun penggunaannya bisa dipertimbangkan jika manfaat lebih besar daripada risikonya.",
        "Sertraline tidak boleh dikonsumsi bersamaan dengan obat golongan MAOI dalam 14 hari terakhir.",
        "Penggunaan sertraline perlu hati-hati pada pasien dengan gangguan bipolar, jantung, kejang, stroke, diabetes, atau glaukoma.",
        "Efek samping Sertraline antara lain kantuk, pusing, gangguan konsentrasi, dan risiko interaksi dengan alkohol.",
        "Dosis awal Sertraline untuk depresi adalah 50 mg per hari, dan bisa ditingkatkan hingga maksimal 200 mg tergantung respon pasien.",
        "Untuk gangguan kecemasan sosial, panik, dan PTSD, dosis awal 25 mg dan ditingkatkan jadi 50 mg per hari setelah 1 minggu.",
        "Pada anak usia 6–12 tahun dengan OCD, dosis awal 25 mg/hari, bisa naik ke 50 mg/hari setelah seminggu.",
        "Untuk premenstrual dysphoric disorder, bisa diberikan secara kontinu atau intermiten, dosis mulai dari 50 mg/hari.",
        "Bentuk sediaan Sertraline adalah tablet yang diminum satu kali sehari, pagi atau malam sesuai anjuran dokter.",
        "Pasien perlu rutin memeriksakan diri saat menggunakan Sertraline, terutama di awal pengobatan."
      ]

  )
scorer = ResponseRelevancy(llm=evaluator_llm, embeddings=evaluator_embeddings)
RR_score = scorer.single_turn_score(sample)  
print("Response Relevancy Score:", RR_score)

Response Relevancy Score: 0.7858849933549745


In [10]:
scorer = Faithfulness(llm=evaluator_llm)
F_Score = await scorer.single_turn_ascore(sample)
print("Faithfullness Score:", F_Score)

Faithfullness Score: 0.6363636363636364


In [11]:
scorer = NoiseSensitivity(llm=evaluator_llm)
Noise_Score = await scorer.single_turn_ascore(sample)
print("Noise Sensitivity Score:", Noise_Score)

Noise Sensitivity Score: 0.09090909090909091


In [12]:
context_precision = LLMContextPrecisionWithoutReference(llm=evaluator_llm)
CP_Score = await context_precision.single_turn_ascore(sample)
print("Context Precision Score:", CP_Score)

Context Precision Score: 0.7751945970599071


In [13]:
context_recall = LLMContextRecall(llm=evaluator_llm)
CR_Score = await context_recall.single_turn_ascore(sample)
print("Context Recall Score:", CR_Score)

Context Recall Score: 0.875
